In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pickle
import re
import pandas as pd
from scipy import sparse, io
import matplotlib.pylab as plt
import numpy as np
from glob import glob
from pyspark.sql import SparkSession
from tqdm import tqdm
from collections import defaultdict, Counter
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

RAW_TWEETS_PATH = "/data/dnc2020/raw_tweets/"
PARQUET_TWEET_PATH = '/data/navid/processed_tweets'
PARQUET_USER_PATH = "/data/navid/processed_users"
PARQUET_RTNET_PATH = "/data/kenny/processed_rt_network"

In [2]:
## We use spark to analyze the data efficiently. You may opt for a different format
## Also note we here assume a machine with a reasonably large number of cores (16) 
## and RAM (at least 32 GB). Adjust below accordingly
spark = (
    SparkSession
    .builder
    .master("local[{}]".format(16))
    .config("spark.driver.memory", "{}g".format(50))
    .config("spark.driver.maxResultSize", f"{10}g")
    .getOrCreate()
)


23/07/25 11:05:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/25 11:05:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Write out subset of tweets from Bernie Users
 

In [5]:
all_tweets_df = spark.read.parquet(os.path.join(PARQUET_TWEET_PATH, '*', '*'))

In [16]:
clustering_res = spark.read.csv("/data/dnc2020/bernie_vsp.csv",sep=" ", header=True)

In [17]:
clustering_res.show(5,False)

+----+
|uid |
+----+
|765 |
|1033|
|1084|
|1265|
|1999|
+----+
only showing top 5 rows



In [8]:
cluster_tweets = clustering_res.join(all_tweets_df, ['uid'])

In [9]:
cluster_tweets.write.parquet("/data/dnc2020/tweets_clustering",mode="overwrite")

# Evaluation: RTs of Candidates (generate data)

In [23]:
accts = [("939091", "JoeBiden"),
("1193981789787824129", "Mike2020"),
("226222147", "PeteButtigieg"),
("426028646", "JohnDelaney"),
("26637348", "TulsiGabbard"),
("33537967", "amyklobuchar"),
("61636488", "DevalPatrick"),
("216776631", "BernieSanders"),
("949934436", "TomSteyer"),
("357606935", "ewarren"),
("2228878592", "AndrewYang"),
("4797361833", "TomPerez"),
("1108472017144201216", "TrumpWarRoom"),
("818910970567344128", "VP45")]

In [24]:
spark = (
    SparkSession
    .builder
    .master("local[{}]".format(16))
    .config("spark.driver.memory", "{}g".format(50))
    .config("spark.driver.maxResultSize", f"{20}g")
    .getOrCreate()
)

In [40]:
trimmed_rt_net = spark.read.parquet("data/trimmed_rtnet/")

In [41]:
acct_df = spark.createDataFrame(accts, schema=T.StructType([
                    T.StructField('rt_uid', T.StringType(), True), 
                   T.StructField('sn', T.StringType(), True)]))

In [42]:
trimmed_rt_net = trimmed_rt_net.join(acct_df, "rt_uid")

In [43]:
trimmed_rt_net.write.parquet("rtnet_eval",mode='overwrite')

# Evaluation: Most Common Identities Per Cluster

In [12]:
user_desc_df = spark.read.parquet("/data/dnc2020/user_descript")

In [ ]:
from personal_identifiers import *

@F.udf(returnType=T.ArrayType(T.StringType()))
def extract_pid(bio):
    if bio is None:
        return ""
    return find_identifiers_simple(bio)[0]

user_to_pi = (user_desc_df
 .withColumn('pi', extract_pid('description'))
 .select(F.col("id"),F.explode(F.col("pi")))
 .dropDuplicates(subset=['id','col'])
)

user_to_pi.write.parquet("/data/dnc2020/user_pi_net", mode="overwrite")

In [4]:
user_to_pi = spark.read.parquet("/data/dnc2020/user_pi_net/")

In [5]:
clustering_res = spark.read.csv("bernie_vsp_yz.csv",sep=" ",
                                header=True)
clustering_res.show(5,False)

+----+---+--------+
|uid |grp|utype   |
+----+---+--------+
|765 |27 |ordinary|
|1033|11 |ordinary|
|1084|11 |ordinary|
|1265|11 |ordinary|
|1999|3  |ordinary|
+----+---+--------+
only showing top 5 rows



In [6]:
user_to_pi.show(5, False)

+--------+-----------------------+
|id      |col                    |
+--------+-----------------------+
|591     |build your brand online|
|1904551 |mama                   |
|9507972 |espace2                |
|12517802|action                 |
|14121921|commercial voice talent|
+--------+-----------------------+
only showing top 5 rows



In [6]:
mg = user_to_pi.join(clustering_res,
                     user_to_pi.id == clustering_res.uid)

In [7]:
mg.groupby(["col","grp","utype"]).count().write.parquet("bern_pi",mode="overwrite")

In [8]:
spark.stop()

In [7]:
df = pd.read_parquet("data/bern_pi/")

In [8]:
df.head()

,col,grp,utype,count
0,rutgers english phd: i write without knowing t...,27,ordinary,1
1,dont bite unless asked,5,ordinary,1
2,foreign policy,27,ordinary,16
3,floridian,27,influencer,1
4,live la vida bila loca,27,ordinary,1


In [9]:
import pandas as pd
import numpy as np

"""Fx to get weighted log odds"""
def get_weighted_log_odds(dataFrame, groupSet, featureName, nCount):
    """
    Fx to get weighted log odds ratio of features depending on their grouping.
    Adapted from https://github.com/juliasilge/tidylo/blob/main/R/bind_log_odds.R
    Input is a dataframe that has columns of your set (groups you are comparing to one another),
    feature (the thing you want to see the weighted log odds of) and the count (n) of
    features per set. This is flexible enough to get weighted log odds of any set, feature
    and count combination.
    """
    alphas = dataFrame.groupby(featureName).sum(nCount).reset_index()[[featureName, nCount]].rename(columns={nCount:'alpha'})  # Get total # of features across whole dataset
    dataFrame_II = dataFrame.merge(alphas, on= featureName, how = 'left')
    dataFrame_II['y_wi'] = dataFrame_II[nCount] + dataFrame_II['alpha']  # "y_wi is the pseudo count of word w in group i"
    y_ws = dataFrame_II.groupby(featureName).sum('y_wi').reset_index()[[featureName, 'y_wi']].rename(columns={'y_wi': 'y_w'})  # "y_w is the total count of word w"
    # Actually is sort of an exaggerated count ¯\_(ツ)_/¯
    dataFrame_III = dataFrame_II.merge(y_ws, on=featureName, how='left')
    # Do exaggerated count per set:
    n_is = dataFrame_II.groupby(groupSet).sum('y_wi').reset_index()[[groupSet, 'y_wi']].rename(columns={'y_wi': 'n_i'})
    pseudo_counts = dataFrame_III.merge(n_is, on= groupSet, how = 'left')  # Combine everything together! Now ready to do weighted log odds calc
    pseudo_counts['omega_wi'] = pseudo_counts['y_wi'] / (pseudo_counts['n_i'] - pseudo_counts['y_wi'])  # # odds in group i
    pseudo_counts['omega_w'] = pseudo_counts['y_w'] / (sum(pseudo_counts['y_wi']) - pseudo_counts['y_w'])  # overall odds
    pseudo_counts['delta_wi'] = (np.log(pseudo_counts['omega_wi'])) - (np.log(pseudo_counts['omega_w']))   # eqn 15 << not sure what this means, it's in OG R code
    pseudo_counts['sigma2_wi'] = 1 / pseudo_counts['y_wi'] + 1 / pseudo_counts['y_w']           # eqn 18
    pseudo_counts['zeta_wi'] = pseudo_counts['delta_wi'] / (np.sqrt(pseudo_counts['sigma2_wi']))       # eqn 21
    return pseudo_counts.rename(columns={'zeta_wi': 'log_odds_weighted', 'delta_wi': 'log_odds'}).sort_values('log_odds_weighted', ascending = False)

In [10]:
df.columns = ['col','grp','utype','n']

In [11]:
df_res = get_weighted_log_odds(df[df.utype == "influencer"], "grp","col","n")

In [12]:
df_res.grp.unique()

array(['3', '5', '11', '19', '27'], dtype=object)

In [22]:
for g in df_res.grp.unique():
    print(g + " \t " + ", ".join(df_res[(df_res.grp == g)  ].sort_values(
        "log_odds_weighted",ascending=False).iloc[:25].col) + "")

3 	 bernie2020, he, him, notmeus, she, medicareforall, her, progressive, berniesanders, blacklivesmatter, writer, blm, greennewdeal, gnd, activist, neverbiden, they, host, socialist, organizer, democratic socialist, author, alum, founder, generalstrike
5 	 he, she, him, bernie2020, her, medicareforall, m4a, notmeus, blm, they, blacklivesmatter, writer, medicare4all, 18, fan account, berniesanders, politics, votegreen, 21, artist, acab, demexit, them, neverbidennevertrump, 22
11 	 outline, staff writer, deadspin, curaffairs, newrepublic, writer, bylines, gmail, nytimes, vice, horror, gqmagazine, tv writer, tim, trashfuturepod, paste, eattherichpod, any, rip, labor, thebafflermag, listen to, podcasts, jewishcurrents, researcher
19 	 nomiddleground, bernie, green, peoplesparty, freecollege, freeassange, independent voter, bernieontheballot, ventura2020, tenforjustice, ibelievetarareade, votegreen, m4all, gogreen, nevertrumpneverbiden, end the wars, software engineer, bernietogreen, solida